# StyleGAN3 お試しコード 

## 簡単に使ってみる。

・NVIDIAの公式リポジトリをクローン

In [ ]:
!git clone https://github.com/NVlabs/stylegan3.git
%cd stylegan3
!pip install Ninja

!nvcc -V

import torch
print(torch.__version__)
print("CUDA is available:",torch.cuda.is_available())

・NVIDIAが提供している、公式の事前学習済みモデルをダウンロード

In [ ]:
import urllib.request

#@markdown 事前学習済みモデル
model="stylegan3-r-afhqv2-512x512" #@param["stylegan3-r-ffhq-1024x1024", "stylegan3-t-ffhq-1024x1024", "stylegan3-r-ffhqu-1024x1024", "stylegan3-t-ffhqu-1024x1024", "stylegan3-r-afhqv2-512x512", "stylegan3-t-afhqv2-512x512", "stylegan3-r-metfaces-1024x1024", "stylegan3-t-metfaces-1024x1024"]
url="https://api.ngc.nvidia.com/v2/models/nvidia/research/stylegan3/versions/1/files/"+model+".pkl"
urllib.request.urlretrieve(url, model)

path="./"+model

・モデル読み込み

In [ ]:
import os
import numpy as np
from PIL import Image
import torch
import pickle
import matplotlib.pyplot as plt

os.makedirs("./out/image/", exist_ok=True)
os.makedirs("./out/w/", exist_ok=True)

with open(path, 'rb') as f:
    G = pickle.load(f)['G_ema'].cuda()#Generator

・画像生成

In [ ]:
#@markdown シード値
seed=1500 #@param{type:"slider", min:0, max:3000, step:1}

z = torch.from_numpy(np.random.RandomState(seed).randn(1, G.z_dim)).cuda()#シード値→潜在ベクトルz
#print(z)
c = None

w = G.mapping(z, c, truncation_psi=0.5, truncation_cutoff=8)#潜在ベクトルz→潜在空間wに拡張
#print(w)

w_npy = w.to('cpu').detach().numpy().copy()#Tensorをnumpyに変換
np.save('./out/w/w_%s'%seed, w_npy)#wを保存
w.cuda()

#画像生成
img = G.synthesis(w, noise_mode='const', force_fp32=True)
img = (img.permute(0, 2, 3, 1) * 127.5 + 128).clamp(0, 255).to(torch.uint8)
Image.fromarray(img[0].cpu().numpy(), 'RGB').save('./out/image/test_%s.png'%seed)#画像保存

#画像表示
plt.figure(figsize=(10,10))
img = Image.open('./out/image/test_%s.png'%seed)
plt.imshow(img)
plt.axis('off')

・潜在空間wの中身を見てみる

In [ ]:
npw=np.load('./out/w/w_%s.npy'%seed)
print(npw)
print("w size:",npw.shape)

## 出力結果のダウンロード

In [ ]:
# 出力結果をまとめたフォルダを圧縮
!zip -r ./stylegan3_output.zip ./out/

# 圧縮したzipのダウンロード
from google.colab import files
files.download("./stylegan3_output.zip")

## 参考文献



*   『 Alias-Free Generative Adversarial Networks (StyleGAN3) | NVIDIA 』(URL: https://nvlabs.github.io/stylegan3/ )
*   『 NVlabs/stylegan3 | GitHub 』(URL: https://github.com/NVlabs/stylegan3 )

